In [1]:
%load_ext autoreload
%autoreload 2

import sys; print('Python %s on %s' % (sys.version, sys.platform))

home_or_office = "home_desktop"

import sys
if home_or_office =="home_desktop":
    sys.path.extend(['C:\\Code\\Libraries', 'C:/Code/Libraries'])
elif home_or_office == "home_laptop":
    sys.path.extend(['C:\\Users\\HKGL003\\Documents\\GitHub\\Libraries'])
else:
    sys.path.extend(['C:\\Users\\troy\\Documents\\Code\\Libraries', 'C:\\Users\\troy\\Documents\\Excel Sheets', 
                     'C:/Users/troy/Documents/Code/Libraries'])


from datetime import date, datetime, timedelta
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt

import QuantLib as ql
import dash_bootstrap_components as dbc
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.express as px

from tia.bbg import LocalTerminal

from FinAPI.BondFut.BondFuture_quantlib import USBondFuturesSingleCTDPricerQL

Python 3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)] on win32


### Snippets

In [5]:
# Common code snippets

# -------------------------------------------------------------------------------------------------
# profiling a function
# -------------------------------------------------------------------------------------------------

import cProfile, pstats

def main():
    # put your code here
    return objFut.skip_adding_live_value_to_history().fwd_yield_history

profiler = cProfile.Profile()
profiler.enable()
main()
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()

# -------------------------------------------------------------------------
# Supress exceptions silently
# -------------------------------------------------------------------------

from contextlib import suppress

def func_which_can_raise_exception(x):
    if x is not None:
        return 1
    else:
        raise Exception("x is None")

def f(x):
    with suppress(Exception): # can be more specfic with the exception type
        func_which_can_raise_exception(x)
v = f(None)  # due to exception, the return value will be None
v is None

         1244 function calls (1223 primitive calls) in 0.014 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.014    0.007 c:\ProgramData\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3424(run_code)
        2    0.000    0.000    0.014    0.007 {built-in method builtins.exec}
        1    0.000    0.000    0.014    0.014 C:\Users\tamaghna\AppData\Local\Temp\ipykernel_13424\4108189913.py:1(<module>)
        1    0.000    0.000    0.014    0.014 C:\Users\tamaghna\AppData\Local\Temp\ipykernel_13424\4108189913.py:6(main)
        1    0.001    0.001    0.014    0.014 C:\Code\Libraries\FinAPI\BondFut\BondFuture_quantlib.py:631(fwd_yield_history)
      4/3    0.000    0.000    0.005    0.002 c:\ProgramData\anaconda3\lib\site-packages\pandas\core\series.py:342(__init__)
        1    0.000    0.000    0.004    0.004 c:\ProgramData\anaconda3\lib\site-packages\pandas\core\series.py:486(

True

### Guide to features in Version 1

1. Ability to Compute yield, dv01
2. Historical CTDs and historical yields
3. Works for both historical Futures and Current Futures
4. Able to persist necessary data


In [42]:
objFut = USBondFuturesSingleCTDPricerQL("TYU13")

##### MISC

In [11]:
# Convert between CuSIP and ISIN 
import string 

def cusip_checksum(cusip_8char):
        """
        Determines the last digit of the 9 digit Cusip based on the 1st 8 characters. A full Cusip code 
        consists of 9 digits. 
            - The first six characters are known as the base (or CUSIP-6), and identify the issuer (company)
            - The seventh and eighth digit identify the exact issue or instrument.
            - The ninth digit is an automatically generated checksum based on the first eight digits.
        
        The checksum is generated using Luhn algorithm
        """
        cusipchars = string.digits + string.ascii_uppercase + "*@#"
        cusip2value = dict((ch, n) for n,ch in enumerate(cusipchars))
        cusipweights = [1,2,1,2,1,2,1,2]
        digits = [(w*cusip2value[ch]) for w,ch in zip(cusipweights, cusip_8char)]
        cs = sum([(x%10+x//10) for x in digits]) % 10
        return str((10-cs)%10)

def isin_checksum(code):
    get_numerical_code = lambda c: str(ord(c) - 55)
    encode_letters = lambda c: c if c.isdigit() else get_numerical_code(c)
    to_digest = ''.join(map(encode_letters, code))

    ints = [int(s) for s in to_digest[::-1]]
    every_second_doubled = [x * 2 for x in ints[::2]] + ints[1::2]

    sum_digits = lambda i: sum(divmod(i,10))
    digit_sum = sum([sum_digits(i) for i in every_second_doubled])

    check_digit = (10 - digit_sum % 10) % 10
    return check_digit

def get_isin_from_cusip(cusip_str, country_code):
    """
    >>> get_isin_from_cusip('037833100', 'US')
    'US0378331005'
    """
    isin_to_digest = country_code + cusip_str.upper()
    check_digit = isin_checksum(isin_to_digest)
    return isin_to_digest + str(check_digit)

cusip_checksum("912828ZQ"), get_isin_from_cusip("912828ZQ6", "US"), 'US912828ZQ64'[2:-1], objFut.curr_ctd_obj.get_isin_from_cusip("912828ZQ", "US")

('6', 'US912828ZQ64', '912828ZQ6', 'US912828ZQ64')

##### Pricing Snapshot

In [43]:
# Test for prcing snapshot - LIVE
objFut.pricing_snapshot()

[2023-08-25 01:02:13,785] DEBUG [HistoryManager] [HistoryManager.py] [read_ticker_ts][118] [PID:13424 TID:25556] Cannot find the file C:\Code\Libraries\FinAPI\HistoricalData\Data\AdhocHistory\TYU13_PX_OPEN.csv corresponding to the ticker TYU13 COMDTY
[2023-08-25 01:02:13,814] ERROR [HistoryMaager] [HistoryMaager.py] [get_history][582] [PID:13424 TID:25556] 'NoeType' object has o attribute 'ame'
'Traceback (most recet call last):\  File "C:\\Code\\Libraries\\FiAPI\\HistoricalData\\HistoryMaager.py", lie 558, i get_history\    ts.ame = ticker.split()[0]\AttributeError: \'NoeType\' object has o attribute \'ame\''
[2023-08-25 01:02:14,093] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][413] [PID:13424 TID:25556] BloombergHistory: Updating History for bbg ticker: TYU13 COMDTY
[2023-08-25 01:02:14,095] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][433] [PID:13424 TID:25556] --- using def_start , start and end dates as 1901-01-01 00:00:00, None and 2023-08-2

{'PX_OPEN': 126.796875,
 'PX_HIGH': 126.921875,
 'PX_LOW': 126.109375,
 'PX_LAST': 126.125,
 'PX_VOLUME': 13339.0,
 'OPEN_INT': 20932.0,
 'FUT_CTD_CUSIP': '912828ND',
 'FUT_CNVS_FACTOR': 0.8671,
 'FUT_IMPLIED_REPO_RT': -1.8703,
 'FUT_ACTUAL_REPO_RT': 0.05857,
 'CONVENTIONAL_CTD_FORWARD_FRSK': 7.5007,
 'VALUE_DT': datetime.datetime(2013, 9, 19, 0, 0),
 'PRICE': 126.125}

In [13]:
# Test for prcing snapshot - DATED
data_dt = datetime(2023, 8, 11)
objFut.pricing_snapshot(data_dt)

[2023-08-19 01:54:32,140] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][413] [PID:13988 TID:19996] BloombergHistory: Updating History for bbg ticker: TYU3 COMDTY
[2023-08-19 01:54:32,196] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][433] [PID:13988 TID:19996] --- using def_start , start and end dates as 2022-12-21 00:00:00, 2023-01-04 00:00:00 and 2023-08-19
[2023-08-19 01:54:32,198] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][477] [PID:13988 TID:19996] Data Present, trying to download missing data before the cached start_date - start and end dates as 2022-12-21 00:00:00, 2023-01-04 00:00:00
[2023-08-19 01:54:32,199] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][485] [PID:13988 TID:19996] No missing data before the cached start
[2023-08-19 01:54:33,392] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][514] [PID:13988 TID:19996] Data Present, trying to download data after the cached end date - star

{'PX_OPEN': 110.671875,
 'PX_HIGH': 110.90625,
 'PX_LOW': 110.125,
 'PX_LAST': 110.1875,
 'PX_VOLUME': 1566489.0,
 'OPEN_INT': 4823038.0,
 'FUT_CTD_CUSIP': '91282CHF',
 'FUT_CNVS_FACTOR': 0.8804,
 'FUT_IMPLIED_REPO_RT': 4.22256,
 'FUT_ACTUAL_REPO_RT': 5.423,
 'CONVENTIONAL_CTD_FORWARD_FRSK': 6.4196,
 'VALUE_DT': datetime.datetime(2023, 8, 11, 0, 0),
 'PRICE': 110.1875}

##### Calcualting Yield, fwd dv01 and spot dv01

In [14]:
# Test yield Pricing
# TYU3 
#   109-28+ = 4.3156
#   110 = 4.2985
#   111 = 4.1433
#   109 = 4.4554

objFut.yld(109+28.5/32) * 100, objFut.yld(110) * 100, objFut.yld(111) * 100, objFut.yld(109) * 100

(4.315609355695778, 4.298528252769028, 4.143264204723527, 4.455435529266129)

In [45]:
# Test DV01 
#  TYU3
#   109-28+ = [6.4, 6.492]
objFut.fwd_dv01(109+28.5/32), objFut.spot_dv01(109+28.5/32, val_dt = objFut.fut_last_trade_dt)

(6.401719220723056, 6.423877057670288)

##### Simple HIstory - Future Fields (Candle, OI, Volume, CUSIPS, CNVS Fact, Imp Repo, Actual Repo), Curr CTD Fields (Price, Yield)

In [16]:
# Test hist Values Price an 

objFut.add_live_value_to_history
ts = objFut.get_hist_values(objFut.futures_ticker, "PX_LAST")
if ts is not None: 
    if objFut.add_live_value_to_history:
        snapshot = objFut.pricing_snapshot()
        ts.loc[objFut.today] = snapshot["PX_LAST"]
objFut.add_live_value_to_history, pd.DataFrame({"manual": ts, "object_method" : objFut.price_history})

(True,
                 manual  object_method
 date                                 
 2022-12-20  114.843750     114.843750
 2022-12-21  114.875000     114.875000
 2022-12-22  114.734375     114.734375
 2022-12-23  114.343750     114.343750
 2022-12-27  113.546875     113.546875
 ...                ...            ...
 2023-08-14  110.000000     110.000000
 2023-08-15  109.765625     109.765625
 2023-08-16  109.562500     109.562500
 2023-08-17  109.281250     109.281250
 2023-08-18  109.687500     109.687500
 
 [167 rows x 2 columns])

##### Computed History - CTD objects, CTD Yield Stiched, CTD DV01 Stched, Future Yield, Future DV01, Gross Basis, Net Basis, Imp Repo

In [27]:
# Test HIstorical CTD Cusips
fld = "FUT_CTD_CUSIP"
ts = objFut.get_hist_values(objFut.futures_ticker, fld)
if ts is not None: 
    if objFut.add_live_value_to_history:
        snapshot = objFut.pricing_snapshot()
        ts.loc[objFut.today] = snapshot[fld]
ts.align(objFut.price_history, join='right')[0].fillna(method='ffill').fillna(method='bfill')
df = pd.DataFrame({"Price": objFut.price_history, "CTD": objFut.ctd_cusip_history, "CTD_manual": ts})

In [3]:
objFut.curr_ctd_price_history

date
2023-05-25     99.281250
2023-05-26     99.273438
2023-05-29     99.273438
2023-05-30     99.992188
2023-05-31    100.265625
                 ...    
2023-08-16     96.406250
2023-08-17     96.328125
2023-08-18     96.453125
2023-08-21     95.968750
2023-08-22     96.046875
Name: US91282CHF14, Length: 64, dtype: float64

In [40]:
objFut.fwd_yield_candles

[2023-08-25 00:54:21,069] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][413] [PID:13424 TID:25556] BloombergHistory: Updating History for bbg ticker: TYU3 COMDTY
[2023-08-25 00:54:21,086] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][433] [PID:13424 TID:25556] --- using def_start , start and end dates as 2022-12-21 00:00:00, 2023-01-04 00:00:00 and 2023-08-25
[2023-08-25 00:54:21,087] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][477] [PID:13424 TID:25556] Data Present, trying to download missing data before the cached start_date - start and end dates as 2022-12-21 00:00:00, 2023-01-04 00:00:00
[2023-08-25 00:54:21,089] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][485] [PID:13424 TID:25556] No missing data before the cached start
[2023-08-25 00:54:22,206] DEBUG [HistoryManager] [HistoryManager.py] [update_history_csv][514] [PID:13424 TID:25556] Data Present, trying to download data after the cached end date - star

False
False
False
False
False
False


,OPEN,HIGH,LOW,CLOSE,VOLUME,OPEN_INT
2022-12-20,NaN,NaN,NaN,0.037096,NaN,NaN
2022-12-21,NaN,NaN,NaN,0.037053,NaN,NaN
2022-12-22,NaN,NaN,NaN,0.037246,NaN,NaN
2022-12-23,NaN,NaN,NaN,0.037782,NaN,NaN
2022-12-27,NaN,NaN,NaN,0.038881,NaN,NaN
...,...,...,...,...,...,...
2023-08-17,0.043694,0.043547,0.044382,0.044111,1741118.0,4738353.0
2023-08-18,0.043792,0.043156,0.043866,0.043547,1251625.0,4701121.0
2023-08-21,0.043547,0.043474,0.044604,0.044431,1514677.0,4583891.0
2023-08-22,0.044431,0.044111,0.044752,0.044357,1396508.0,4528694.0


In [38]:
objFut.add_live_value_to_history = False
computed_hist_dict = dict(fwd_yield = objFut.skip_adding_live_value_to_history().fwd_yield_history, 
                          spot_dv01 = objFut.skip_adding_live_value_to_history().spot_dv01_history, 
                          fwd_dv01 = objFut.skip_adding_live_value_to_history().fwd_dv01_history,
                          ctd_spot_price = objFut.skip_adding_live_value_to_history().ctd_spot_price_history, 
                          ctd_spot_yield = objFut.skip_adding_live_value_to_history().ctd_spot_yield_history, 
                          curr_ctd_price = objFut.skip_adding_live_value_to_history().curr_ctd_price_history, 
                          curr_ctd_yield = objFut.skip_adding_live_value_to_history().curr_ctd_yield_history
                          )
df = pd.DataFrame(computed_hist_dict)

True
True
True
True
True


### Sratch

In [33]:
from FinAPI.HistoricalData.HistoryManager import BloombergHistory

# df = BloombergHistory.get_intradaybar_history(objFut.futures_ticker, 'TRADE', 1,
#                                                      def_start=objFut.static_args['FUT_FIRST_TRADE_DT'],
#                                                       update_from_bbg=True)
resp = LocalTerminal.get_intraday_bar(objFut.futures_ticker, event='TRADE', start=objFut.static_args['FUT_FIRST_TRADE_DT'], end=None, 
                                      interval=1)
df2 = resp.as_frame()


In [ ]:
from tia.bbg import LocalTerminal, BbgDataManager

mgr = BbgDataManager()


# Test for hist values
# Test for invidual historical properties
# Test for intraday candles


({'PX_OPEN': 110.171875,
  'PX_HIGH': 110.3125,
  'PX_LOW': 109.953125,
  'PX_LAST': 109.953125,
  'PX_VOLUME': 455371.0,
  'OPEN_INT': 4823982,
  'FUT_CTD_CUSIP': '91282CHF1',
  'FUT_CNVS_FACTOR': 0.8804,
  'FUT_IMPLIED_REPO_RT': 3.956927820778411,
  'FUT_ACTUAL_REPO_RT': 5.424,
  'CONVENTIONAL_CTD_FORWARD_FRSK': 6.403803160683187},
 {'PX_OPEN': 110.171875,
  'PX_HIGH': 110.3125,
  'PX_LOW': 109.953125,
  'PX_LAST': 109.953125,
  'PX_VOLUME': 455371.0,
  'OPEN_INT': 4823982,
  'FUT_CTD_CUSIP': '91282CHF1',
  'FUT_CNVS_FACTOR': 0.8804,
  'FUT_IMPLIED_REPO_RT': 3.956927820778411,
  'FUT_ACTUAL_REPO_RT': 5.424,
  'CONVENTIONAL_CTD_FORWARD_FRSK': 6.403803160683187})

True